### SPLIT AFRICA DATASET 192

I re-ran this notebook until the split was reasonably even between train/validation/test. In a perfect split the `low-med-high rfracs` would all equal 1 and the `block split fracs` would equal the respective `group fracs`.  

RESULTS:

```
-------------------------------------------------------------------------------------
block relative split: 0.7 0.2 0.1 0.2
block split fracs: 0.56 0.16 0.08 0.2
-------------------------------------------------------------------------------------

train group frac: 0.5968855788761002
* low-med-high rfracs: 1.0010590834202162 1.0090326541013923 0.9630740227527372

valid group frac: 0.16086662153012865
* low-med-high rfracs: 1.0470768560307566 0.9802220497192033 1.0241252203165399

test group frac: 0.07217332430602573
* low-med-high rfracs: 1.0072285870158209 0.9754086104574372 1.2026947754658441

hold_out group frac: 0.17007447528774544
* low-med-high rfracs: 0.9486874350634684 0.997442258185378 1.020758463890597

```

NOTES:

    1. solid split 
    2. the hold out group can be used for additional training or additional validation if needed.
 

---

In [1]:
import pandas as pd
from random import sample

In [2]:
WSIZE=192
DSET=f'surface-water.africa.win{WSIZE}'
NO_DATA_MAX=0.05
IDENTS=['group_id','window_index']

In [3]:
df=pd.read_csv(f'{DSET}.csv')
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=IDENTS).shape[0])
df=df[df.no_data<=NO_DATA_MAX]
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=IDENTS).shape[0])
df.sample(3)

8708 632 8541
7527 600 7385


,gsw_path,s1_path,block_id,group_id,window_index,window,water,not_water,no_data,dataset
1243,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_9_31,group_9.19_31.7,2,"(192, 0, 192, 192)",0.084283,0.915717,0.000000,africa
3740,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_-15_16,group_-15.89_16.1,1,"(0, 192, 192, 192)",0.513699,0.464138,0.022163,africa
2338,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_-1_35,group_-1.23_35.43,2,"(192, 0, 192, 192)",0.041178,0.944038,0.014784,africa


---

In [4]:
def water_counter(df,quantiles=[0.15,0.85],labels=['low','medium','high']):
    wqs=[0]+[df.water.quantile(q) for q in quantiles]+[1+1e-8]
    dfs=[]
    for i,l in enumerate(labels):
        _df=df.copy()[(df.water>=wqs[i])&(df.water<wqs[i+1])]
        _df['water_tag']=l
        dfs.append(_df)
    return pd.concat(dfs).sample(frac=1)


def split_groups(blocks,valid_frac=0.2,test_frac=0.1,hold_out_frac=0.2,relative_frac=True):
    if relative_frac:
        valid_frac=(1-hold_out_frac)*valid_frac
        test_frac=(1-hold_out_frac)*test_frac
    nb_blocks=len(blocks)
    nb_valid=int(valid_frac*nb_blocks)    
    nb_test=int(test_frac*nb_blocks)
    nb_hold_out=int(hold_out_frac*nb_blocks)
    blocks=sample(blocks,nb_blocks)
    valid=blocks[:nb_valid]
    test=blocks[nb_valid:nb_valid+nb_test]
    hold_out=blocks[nb_valid+nb_test:nb_valid+nb_test+nb_hold_out]
    train=blocks[nb_valid+nb_test+nb_hold_out:]
    rtotal=len(train)+len(valid)+len(test)
    print('block relative split:',len(train)/rtotal,len(valid)/rtotal,len(test)/rtotal,len(hold_out)/nb_blocks)
    print('block split fracs:',len(train)/nb_blocks,len(valid)/nb_blocks,len(test)/nb_blocks,len(hold_out)/nb_blocks)
    return train, valid, test, hold_out


def sub_df(typ,df,blocks,cnt):
    
    sdf=df.copy()[df.block_id.isin(blocks)]
    sdf['data_split']=typ
    
    sgcnt=sdf.drop_duplicates(subset=IDENTS).shape[0]
    frac=sgcnt/cnt
    
    ltcnt=df[df.water_tag=='low'].shape[0]*frac
    mtcnt=df[df.water_tag=='medium'].shape[0]*frac
    htcnt=df[df.water_tag=='high'].shape[0]*frac
    
    sltcnt=sdf[sdf.water_tag=='low'].shape[0]
    smtcnt=sdf[sdf.water_tag=='medium'].shape[0]
    shtcnt=sdf[sdf.water_tag=='high'].shape[0]
    print()
    print(f'{typ} group frac:',frac)
    print('* low-med-high rfracs:',sltcnt/ltcnt,smtcnt/mtcnt,shtcnt/htcnt)
    return sdf
    
    
def df_splitter(df,train, valid, test, hold_out):
    cnt=df.drop_duplicates(subset=IDENTS).shape[0]
    tdf=sub_df('train',df,train,cnt)    
    vdf=sub_df('valid',df,valid,cnt)    
    sdf=sub_df('test',df,test,cnt)    
    hdf=sub_df('hold_out',df,hold_out,cnt)
    df=pd.concat([tdf, vdf, sdf, hdf])
    return df

---

In [5]:
df=water_counter(df)

In [6]:
blocks=list(df.block_id.unique())

In [7]:
print('-'*85)
train, valid, test, hold_out=split_groups(blocks)
print('-'*85)
df=df_splitter(df, train, valid, test, hold_out)

-------------------------------------------------------------------------------------
block relative split: 0.7 0.2 0.1 0.2
block split fracs: 0.56 0.16 0.08 0.2
-------------------------------------------------------------------------------------

train group frac: 0.5968855788761002
* low-med-high rfracs: 1.0010590834202162 1.0090326541013923 0.9630740227527372

valid group frac: 0.16086662153012865
* low-med-high rfracs: 1.0470768560307566 0.9802220497192033 1.0241252203165399

test group frac: 0.07217332430602573
* low-med-high rfracs: 1.0072285870158209 0.9754086104574372 1.2026947754658441

hold_out group frac: 0.17007447528774544
* low-med-high rfracs: 0.9486874350634684 0.997442258185378 1.020758463890597


---

In [8]:
df.sample(3)

,gsw_path,s1_path,block_id,group_id,window_index,window,water,not_water,no_data,dataset,water_tag,data_split
6014,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_29_-28,group_29.17_-28.68,3,"(192, 192, 192, 192)",0.282823,0.708496,0.008681,africa_mtn,high,train
3512,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_19_-33,group_19.16_-33.25,3,"(192, 192, 192, 192)",0.026937,0.960097,0.012967,africa,medium,valid
6940,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_29_-31,group_29.22_-31.36,0,"(0, 0, 192, 192)",0.034478,0.962972,0.002550,africa_mtn,medium,hold_out


In [9]:
file_name=f'{DSET}.split.csv'
df.to_csv(file_name,index=False)
print(f'UPLOAD: gsutil cp {file_name} gs://surface-water-public/data/v1/datasets')

UPLOAD: gsutil cp surface-water.africa.win192.split.csv gs://surface-water-public/data/v1/datasets


---

In [14]:
# !gsutil cp surface-water.africa.win192.split.csv gs://surface-water-public/data/v1/datasets

Copying file://surface-water.africa.win192.split.csv [Content-Type=text/csv]...
/ [1 files][  2.4 MiB/  2.4 MiB]                                                
Operation completed over 1 objects/2.4 MiB.                                      


---

##### WATER/NOT RATIO

In [11]:
_df=df[df.data_split.isin(['train','valid'])]

In [12]:
_df.not_water.mean()/_df.water.mean()

8.281917177128546

In [13]:
_df.drop_duplicates(subset=IDENTS)[['water','not_water']].describe()

,water,not_water
count,5596.000000,5596.000000
mean,0.107023,0.888447
std,0.125526,0.126362
min,0.005018,0.374837
25%,0.021939,0.850932
50%,0.055759,0.939155
75%,0.143745,0.974101
max,0.599609,0.994982
